In [3]:
import nest_asyncio

nest_asyncio.apply()

In [4]:
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex
from llama_index.core.response.pprint_utils import pprint_response
from llama_index.llms.openai import OpenAI

from llama_index.core.tools import QueryEngineTool, ToolMetadata
from llama_index.core.query_engine import SubQuestionQueryEngine

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/shane/Documents/Github/ai-
[nltk_data]     exploration/.venv/lib/python3.11/site-
[nltk_data]     packages/llama_index/core/_static/nltk_cache...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/shane/Documents/Github/ai-
[nltk_data]     exploration/.venv/lib/python3.11/site-
[nltk_data]     packages/llama_index/core/_static/nltk_cache...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [5]:
from llama_index.core import Settings

Settings.llm = OpenAI(temperature=0.2, model="gpt-3.5-turbo")

# Download Data

In [7]:
!mkdir -p 'data/10q/'
!wget 'https://raw.githubusercontent.com/run-llama/llama_index/main/docs/examples/data/10q/uber_10q_march_2022.pdf' -O 'data/10q/uber_10q_march_2022.pdf'
!wget 'https://raw.githubusercontent.com/run-llama/llama_index/main/docs/examples/data/10q/uber_10q_june_2022.pdf' -O 'data/10q/uber_10q_june_2022.pdf'
!wget 'https://raw.githubusercontent.com/run-llama/llama_index/main/docs/examples/data/10q/uber_10q_sept_2022.pdf' -O 'data/10q/uber_10q_sept_2022.pdf'

--2024-02-14 21:27:19--  https://raw.githubusercontent.com/run-llama/llama_index/main/docs/examples/data/10q/uber_10q_march_2022.pdf
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1260185 (1.2M) [application/octet-stream]
Saving to: ‘data/10q/uber_10q_march_2022.pdf’

data/10q/uber_10q_m 100%[===================>]   1.20M  7.36MB/s    in 0.2s    

2024-02-14 21:27:20 (7.36 MB/s) - ‘data/10q/uber_10q_march_2022.pdf’ saved [1260185/1260185]

--2024-02-14 21:27:20--  https://raw.githubusercontent.com/run-llama/llama_index/main/docs/examples/data/10q/uber_10q_june_2022.pdf
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|1

In [8]:
# Read the three PDF files
march_2022 = SimpleDirectoryReader(
    input_files=["./data/10q/uber_10q_march_2022.pdf"]
).load_data()
june_2022 = SimpleDirectoryReader(
    input_files=["./data/10q/uber_10q_june_2022.pdf"]
).load_data()
sept_2022 = SimpleDirectoryReader(
    input_files=["./data/10q/uber_10q_sept_2022.pdf"]
).load_data()

In [10]:
# Build indices
march_index = VectorStoreIndex.from_documents(march_2022)
june_index = VectorStoreIndex.from_documents(june_2022)
sept_index = VectorStoreIndex.from_documents(sept_2022)


In [11]:
# Build query engines
march_engine = march_index.as_query_engine(similarity_top_k=3)
june_engine = june_index.as_query_engine(similarity_top_k=3)
sept_engine = sept_index.as_query_engine(similarity_top_k=3)

In [12]:
query_engine_tools = [
    QueryEngineTool(
        query_engine=sept_engine,
        metadata=ToolMetadata(
            name="sept_22",
            description=(
                "Provides information about Uber quarterly financials ending"
                " September 2022"
            ),
        ),
    ),
    QueryEngineTool(
        query_engine=june_engine,
        metadata=ToolMetadata(
            name="june_22",
            description=(
                "Provides information about Uber quarterly financials ending"
                " June 2022"
            ),
        ),
    ),
    QueryEngineTool(
        query_engine=march_engine,
        metadata=ToolMetadata(
            name="march_22",
            description=(
                "Provides information about Uber quarterly financials ending"
                " March 2022"
            ),
        ),
    ),
]

In [13]:
s_engine = SubQuestionQueryEngine.from_defaults(
    query_engine_tools=query_engine_tools
)

# Run queries

In [14]:
response = s_engine.query(
    "Analyze Uber revenue growth over the latest two quarter filings"
)


Generated 2 sub questions.
[sept_22] Q: What is the revenue for Uber in the September 2022 quarter?
[june_22] Q: What is the revenue for Uber in the June 2022 quarter?
[june_22] A: The revenue for Uber in the June 2022 quarter is $8.1 billion.
[sept_22] A: The revenue for Uber in the September 2022 quarter is $8.3 billion.


In [15]:
print(response)

Uber's revenue has shown growth over the latest two quarter filings. In the September 2022 quarter, the revenue was $8.3 billion, while in the June 2022 quarter, the revenue was $8.1 billion. This indicates a positive trend in revenue growth for Uber.


In [18]:
response = s_engine.query(
    "Analyze change in macro environment over the 3 quarters"
)

Generated 3 sub questions.
[sept_22] Q: What were the key financial metrics for Uber in September 2022?
[june_22] Q: What were the key financial metrics for Uber in June 2022?
[march_22] Q: What were the key financial metrics for Uber in March 2022?
[march_22] A: In March 2022, Uber reported the following key financial metrics:
- Revenue: $6.9 billion, representing a year-over-year increase of 136%.
- Gross Bookings: $26.4 billion, with a year-over-year growth of 39% on a constant currency basis.
- Adjusted EBITDA: $168 million, showing an improvement of $527 million compared to the same period in 2021.
- Net loss attributable to Uber Technologies, Inc.: $5.9 billion, which includes an unrealized loss on debt and equity securities of $5.6 billion.
- Cash and cash equivalents: $4.2 billion at the end of the quarter.
[june_22] A: In June 2022, Uber reported revenue of $8.1 billion, which represented a 105% increase compared to the same period in 2021. The overall Gross Bookings for the c

In [19]:
print(response)

The change in the macro environment over the three quarters can be analyzed based on the key financial metrics reported by Uber. In September 2022, Uber experienced significant growth in revenue, with a 72% increase compared to the same period in 2021. This indicates a positive trend in the company's financial performance. Additionally, the overall Gross Bookings increased by 32% on a constant currency basis, reflecting increased demand for Uber's services.

In June 2022, Uber continued to show strong growth in revenue, with a 105% increase compared to the same period in 2021. The overall Gross Bookings also increased by 36% on a constant currency basis. However, the company reported a net loss attributable to Uber Technologies, Inc. of $2.6 billion, which included unrealized losses on debt and equity securities. This suggests that while Uber's revenue and Gross Bookings increased, the company faced challenges in managing its expenses.

In March 2022, Uber experienced significant year-

In [20]:
response = s_engine.query("How much cash did Uber have in sept 2022")


Generated 1 sub questions.
[sept_22] Q: What is the cash balance of Uber in September 2022?
[sept_22] A: The cash balance of Uber in September 2022 is $4,865 million.


In [21]:
print(response)


Uber had $4,865 million in cash in September 2022.
